<a href="https://colab.research.google.com/github/vedanthshenoy/Sales-Analysis-Prediction-and-Recommendation/blob/main/shop_data_recommendation_using_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
data=pd.read_csv('/content/drive/MyDrive/shopdata.csv',encoding= 'unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [30]:
data.drop(columns=['InvoiceNo', 'StockCode', 'UnitPrice', 'Country','InvoiceDate'],inplace=True)
data=data.dropna(how='any')
buys = pd.DataFrame(data.groupby('CustomerID')['Quantity'].mean())  
buys['Total Quantity'] = pd.DataFrame(data.groupby('CustomerID')['Quantity'].count()) 
data = data.pivot_table(index ='CustomerID', columns ='Description', values ='Quantity') 
data=data.fillna(0)
data.head()

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,...,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [31]:
import numpy as np

U, s, V = np.linalg.svd(data)

The 's' matrix using this numpy svd is being obtained as a vector of all singular values rather than as a diagonal matrix with diagonal entries as singular values.
So here we convert the vector into a diagonal matrix.

In [32]:
s=np.diag(s)
s

array([[1.25426899e+04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.42866112e+03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.66970788e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.95755535e-13, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.95755535e-13, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.95755535e-13]])

Again, whatever we are reffering as V is actually V transpose.

Now we don't need all the components. Let's say we take the most important  features.

In [33]:
n_components=4
U=U[:,:n_components]
s=s[:n_components,:n_components]
V=V[:n_components,:]

We can get back the data matrix(almost) with all values filled from the U,s and V matrices.
The more components we take the more nearer the X matrix will be to our original data matrix.

In [35]:
X=np.matmul(U,s)
X=np.matmul(X,V)
X=pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3856,3857,3858,3859,3860,3861,3862,3863,3864,3865,3866,3867,3868,3869,3870,3871,3872,3873,3874,3875,3876,3877,3878,3879,3880,3881,3882,3883,3884,3885,3886,3887,3888,3889,3890,3891,3892,3893,3894,3895
0,5.159855e-17,4.191729e-15,7.902460e-15,1.062579e-15,1.759989e-17,1.487078e-17,1.174831e-16,4.079194e-15,3.684607e-15,5.387337e-16,1.389953e-18,3.895202e-16,5.810114e-15,6.521390e-16,8.416617e-17,2.449891e-17,4.382072e-18,1.642539e-16,7.832662e-16,1.317256e-14,8.715819e-15,4.241047e-15,8.777202e-15,1.129930e-15,4.479044e-16,4.193171e-16,5.429878e-17,8.610439e-17,2.001877e-16,1.842985e-18,3.528715e-16,1.919495e-18,1.013184e-15,1.394483e-15,0.0,6.746604e-17,4.778235e-17,6.801190e-17,1.336239e-15,1.824024e-16,...,2.626029e-18,3.730715e-17,1.397304e-18,1.548471e-17,6.182351e-16,2.590228e-15,3.473911e-16,7.929352e-18,1.465394e-17,1.251421e-18,9.467461e-16,6.051949e-18,1.533878e-15,2.641996e-16,4.207369e-18,2.800765e-15,2.698977e-15,9.877736e-17,4.644969e-17,2.813865e-15,3.123305e-16,8.469778e-19,6.805579e-18,3.295541e-18,2.523612e-16,2.964734e-19,7.301374e-17,1.060069e-16,4.552726e-15,4.088152e-18,1.180228e-19,1.247638e-17,1.258629e-16,2.123664e-15,5.981252e-18,3.667167e-15,-2.546280e-19,2.787105e-15,4.760308e-18,1.391078e-16
1,1.411411e-02,9.039583e-01,1.695921e+00,2.307227e-01,3.791033e-03,3.240661e-03,2.670468e-02,8.782523e-01,7.702856e-01,1.332329e-01,3.277767e-04,9.514140e-02,1.293848e+00,1.466833e-01,1.838214e-02,5.575571e-03,9.496525e-04,4.012224e-02,1.827916e-01,2.945933e+00,2.097167e+00,1.128918e+00,1.878993e+00,2.893313e-01,9.766715e-02,9.138532e-02,1.249886e-02,2.105928e-02,4.843286e-02,4.233905e-04,8.418716e-02,4.883760e-04,2.202139e-01,3.472685e-01,0.0,1.630529e-02,1.081602e-02,1.652729e-02,3.016164e-01,4.096033e-02,...,6.121439e-04,8.528699e-03,4.295822e-04,3.473052e-03,1.325185e-01,5.590054e-01,7.267568e-02,1.675240e-03,3.467070e-03,2.802148e-04,1.981405e-01,1.306360e-03,4.202189e-01,5.791159e-02,9.158563e-04,6.053818e-01,5.819371e-01,2.287166e-02,1.019443e-02,6.456336e-01,6.994342e-02,2.173260e-04,1.489771e-03,7.448343e-04,5.683951e-02,1.171755e-04,1.609750e-02,2.454890e-02,9.860015e-01,9.077106e-04,3.186957e-05,2.688866e-03,2.998640e-02,4.599075e-01,1.467697e-03,7.966643e-01,-6.036785e-05,6.135407e-01,1.061133e-03,3.138458e-02
2,2.772235e-02,1.796697e+00,3.371109e+00,4.582817e-01,7.532800e-03,6.434344e-03,5.293494e-02,1.745751e+00,1.533551e+00,2.629240e-01,6.474826e-04,1.879358e-01,2.566765e+00,2.908408e-01,3.650650e-02,1.104977e-02,1.887309e-03,7.923468e-02,3.618172e-01,5.858109e+00,4.160225e+00,2.240445e+00,3.736311e+00,5.700353e-01,1.939371e-01,1.815301e-01,2.475714e-02,4.159245e-02,9.570570e-02,8.386787e-04,1.661734e-01,9.628137e-04,4.374727e-01,6.852985e-01,0.0,3.221775e-02,2.144298e-02,3.264899e-02,5.978886e-01,8.124270e-02,...,1.212006e-03,1.688869e-02,8.405187e-04,6.888336e-03,2.634858e-01,1.110624e+00,1.446969e-01,3.331155e-03,6.858433e-03,5.558940e-04,3.944881e-01,2.596365e-03,8.255521e-01,1.149988e-01,1.819555e-03,1.202722e+00,1.156258e+00,4.529297e-02,2.024032e-02,1.278542e+00,1.387483e-01,4.283243e-04,2.958034e-03,1.476264e-03,1.126910e-01,2.271949e-04,3.195358e-02,4.856725e-02,1.958708e+00,1.801595e-03,6.265572e-05,5.344669e-03,5.929396e-02,9.136152e-01,2.896846e-03,1.582354e+00,-1.194370e-04,1.217639e+00,2.105663e-03,6.222167e-02
3,3.605325e-03,2.295485e-01,4.305622e-01,5.859266e-02,9.624894e-04,8.229822e-04,6.791556e-03,2.230097e-01,1.954221e-01,3.395397e-02,8.337997e-05,2.424314e-02,3.288256e-01,3.728919e-02,4.669312e-03,1.417947e-03,2.412031e-04,1.022261e-02,4.651887e-02,7.496958e-01,5.350545e-01,2.894174e-01,4.770583e-01,7.379761e-02,2.480499e-02,2.321291e-02,3.179649e-03,5.366095e-03,1.233788e-02,1.077004e-04,2.142022e-02,1.245676e-04,5.593075e-02,8.852865e-02,0.0,4.153237e-03,2.750333e-03,4.210444e-03,7.668066e-02,1.041351e-02,...,1.557951e-04,2.168628e-03,1.100516e-04,8.829508e-04,3.

When we correlate a row( in this example when we take row 0) of U matrix with the rows of U matrix, we find how similar are the users and then we can retrive back the user ID from the index.

In [37]:
U=pd.DataFrame(U)
U.corrwith(U.iloc[0], axis=1)

0       1.000000
1       0.842566
2       0.855012
3       0.830302
4       0.878196
          ...   
4367    0.833412
4368    0.842387
4369    0.755796
4370    0.604320
4371    0.849331
Length: 4372, dtype: float64

Now we need to sort this in descending order to check how similar are the other users to user 0. Here we find that user 0 is 100% correlated with user 0 which is obviously the case.

In [43]:
U.corrwith(U.iloc[0], axis=1).sort_values(ascending=False)   #if you make the ascending=True or use default then you find the most uncorrelated users

0       1.000000
2996    0.964735
4106    0.962807
1724    0.959517
3168    0.959453
          ...   
1642         NaN
1809         NaN
3349         NaN
4358         NaN
4363         NaN
Length: 4372, dtype: float64

Here we see users with index 299,4106,1724 and so on are the most similar users to user 0.

Here i give a task to find the most similar items using the same way. Don't change the data matrix, instead play around with the V matrix we have.